# Task

- [x] class, decorator and property
- [x] operator overloading

## Class

这个作业昨天已经做过，复制一遍
- 类和继承，方法
- 属性，私有属性，装饰器

In [1]:
class Animal(object):
    def __init__(self):
        print('animal_init')
    def run(self):
        print('animal run')
        
class Human(Animal):
    def __init__(self):
        super().__init__()
        print('human_init')
    def run(self):
        print('human run')

class Man(Human):
    def __init__(self, name, age, gender):
        super().__init__()
        print('man_init')
        self.__gender = gender    # 演示原生私有变量
        self.__name = name        # 演示property方法
        self.__age = age          # 演示装饰器property
    def run(self):
        super().run()             # 不传参表示最近parent(此处为Human)
        super(Human, self).run()  # 传参表示参数的parent(此处为Animal)
    
    # 定义一个get私有变量的方法，然后传到property方法里去
    def get_name(self):
        return self.__name
    name = property(get_name)
    
    # 观察上面把方法作为参数传进去，不就是装饰器干的事吗？所以：
    @property
    def age(self):
        return self.__age
    
    # setter的语法非常奇怪，
    # 其实就是在已经age = property(f_get)的前提下，对age进行fset的赋值
    # property是可以接一个getter, 一个setter作为参数的
    # 等同于：age = property(get_age, set_age)
    @age.setter
    def age(self, value):
        self.__age = value
        
    num_foot = 2 # <<< 这样就变成类变量（静态变量）了, 【定义】时就已经执行一次了，而非第一次初始化时
    
        
m = Man("walker", 28, 90)
m.run()
print("__形式私有变量:", m._Man__gender) # 打印私有变量的方式，但尽量避免
print("property方法封装:", m.name)
print("装饰器封装，dot语法:", m.age)
m.age = 37
print("after set age=37", m.age)
print(f"type(m.name): {type(m.name)} \ntype(m.age): {type(m.age)}") # 可见类型取决于返回值，而不是函数本身
print("number of foot: (m.num_foot)", m.num_foot) # 当实体变量用
print("number of foot: (Man.num_foot)", Man.num_foot) # 当静态变量用

animal_init
human_init
man_init
human run
animal run
__形式私有变量: 90
property方法封装: walker
装饰器封装，dot语法: 28
after set age=37 37
type(m.name): <class 'str'> 
type(m.age): <class 'int'>
number of foot: (m.num_foot) 2
number of foot: (Man.num_foot) 2


## Operator Overloading

作业要求，设计一个表达式类，支持`常数`，`变量`，以及加减乘除和求导（不做高阶求导）。   
然后用该表达式求$(2*x-6y)/(3*x+4*y)$在x=2, y=3处的的导数值

可见，我们需要重载四个基础运算符（`__add__`,`__sub__`,`__mul__`,`__truediv__`），以及实现求导方法(`drivative`)，还要自定义字符串输出。

In [108]:
class Expr(object):
    def eval(self, **values):     # evaluate the expression
        pass
    def deriv(self, x):           # get the derivative of x
        pass
    
    def __add__(self, other):     # overloading + operator
        return Add(self, other).simplify()
    def __sub__(self, other):     # overloading - operator
        return Sub(self, other).simplify()
    def __mul__(self, other):     # overloading * operator
        return Mul(self, other).simplify()
    def __neg__(self):            # overloading - operator(单目)
        return Neg(self)
    def __truediv__(self, other): # overloading / operator
        return TrueDiv(self, other).simplify()
    
class Const(Expr):
    def __init__(self, value):
        self.value = value
    def eval(self, **values):
        return self.value
    def deriv(self, x):
        return Const(0)
    def __repr__(self):
        return str(self.value)
    
def get_var_name(x):
    return x.name if isinstance(x, Variable) else x
    
class Variable(Expr):
    def __init__(self, name):
        self.name = name
    def eval(self, **kwargs):
        '''
        x.eval(x=3, y=4) ==> 3
        x.eval(y=4) ==> error
        '''
        if self.name in kwargs:
            return kwargs[self.name]
        else:
            raise Exception(f"Variable {self.name} is not found")
    def deriv(self, x):
        '''
        derivative of var itself is 1, 
        otherwise is 0
        '''
        var_name = get_var_name(x)
        return Const(1 if var_name == self.name else 0)
    def __repr__(self):
        return self.name
    
x1 = Variable("x")
print(x1)
print(x1.eval(x=3))
print(x1.deriv("x"))
print(x1.deriv("x").eval(x=4))
print(x1.deriv("y").eval(x=4))

x
3
1
1
0


In [111]:
class Add(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) + self.right.eval(**kwargs)
    def deriv(self, x):
        # get a NEW Add object
        return self.left.deriv(x) + self.right.deriv(x)
    def __repr__(self):
        return f"({self.left} + {self.right})"
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value + right.value)
        return self

class Sub(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) - self.right.eval(**kwargs)
    def deriv(self, x):
        return self.left.deriv(x) - self.right.deriv(x)
    def __repr__(self):
        return f"({self.left} - {self.right})"
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value - right.value)
        return self
    
class Neg(Expr):
    def __init__(self, value):
        self.value = value
    def eval(self, **kwargs):
        return -self.value.eval(**kwargs)
    def deriv(self, x):
        return -self.value.deriv(x)
#     def simplify(self):
#         if isinstance(self.value, Const):
#             return Const(-self.value.value)
#         return self
    def __repr__(self):
        return '(-%s)' % self.value
        
class Mul(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) * self.right.eval(**kwargs)
    def deriv(self, x):
        '''
        (uv)' = u'v + v'u  (Chain Rule)
        '''
        u, v = self.left, self.right
        return u.deriv(x) * v + v.deriv(x) * u
    def __repr__(self):
        return f"({self.left} * {self.right})"
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            return Const(left.value * right.value)
        if left_const:
            if left.value == 0: return Const(0)
            if left.value == 1: return right
        if right_const:
            if right.value == 0: return Const(0)
            if right.value == 1: return left
        return self
    
class TrueDiv(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def eval(self, **kwargs):
        return self.left.eval(**kwargs) / self.right.eval(**kwargs)
    def deriv(self, x):
        '''
        (u/v)' = (u'v - v'u) / v^2 (Quotient Rule)
        '''
        u, v = self.left, self.right
        return (u.deriv(x)*v - u*v.deriv(x))/(v*v)
    def __repr__(self):
        return f"({self.left} / {self.right})"    
    
    def simplify(self):
        left, right = self.left, self.right
        left_const = isinstance(left, Const) 
        right_const = isinstance(right, Const)
        if left_const and right_const:
            if right_const.value == 0:
                raise Error("divide by zero error")
            return Const(left.value / right.value)
        if left_const and left.value == 0: return Const(0)
        if right_const and right.value == 1: return left
        return self
    
c1 = Const(13)
c2 = Variable("a")
c = c1 + c2
d = c1 - c2
e = c1 * c2
f = c1 / c2
g = -c1
h = -c2
print(f'{c} = {c1.eval()} + 5 = {c.eval(a=5)}')
print(f'{d} = {c1.eval()} - 5 = {d.eval(a=5)}')
print(f'{e} = {c1.eval()} * 5 = {e.eval(a=5)}')
print(f'{f} = {c1.eval()} / 5 = {f.eval(a=5)}')
print(f'{g} = {g.eval(a=2)}')
print(f'{h} = {h.eval(a=2)}')
print('-'*24)
print(f'{c}\' = {c.deriv("a").eval(a=3)}')
print(f'{d}\' = {d.deriv("a").eval(a=3)}')
print(f'{e}\' = {e.deriv("a").eval(a=3)}')
print(f'{f}\' = {f.deriv("a").eval(a=3)}')


(13 + a) = 13 + 5 = 18
(13 - a) = 13 - 5 = 8
(13 * a) = 13 * 5 = 65
(13 / a) = 13 / 5 = 2.6
(-13) = -13
(-a) = -2
------------------------
(13 + a)' = 1
(13 - a)' = -1
(13 * a)' = 13
(13 / a)' = -1.4444444444444444


求$(2*x-6y)/(3*x+4*y)$在x=2, y=3处的的导数值

In [112]:
x = Variable("x")
y = Variable("y")
exp1 = Const(2) * x - Const(6) * y
exp2 = Const(3) * x + Const(4) * y
exp = exp1 / exp2
d1 = exp.deriv(x)
d2 = exp.deriv(y)
values = {"x":2, "y":3}
print(exp)
print(f'∂f/∂x = {d1}')
print(f'∂f/∂y = {d2}')
print(values,'----'*10)
print(f'∂f/∂x = {d1.eval(**values)}')
print(f'∂f/∂y = {d2.eval(**values)}')

(((2 * x) - (6 * y)) / ((3 * x) + (4 * y)))
∂f/∂x = (((2 * ((3 * x) + (4 * y))) - (((2 * x) - (6 * y)) * 3)) / (((3 * x) + (4 * y)) * ((3 * x) + (4 * y))))
∂f/∂y = (((-6 * ((3 * x) + (4 * y))) - (((2 * x) - (6 * y)) * 4)) / (((3 * x) + (4 * y)) * ((3 * x) + (4 * y))))
{'x': 2, 'y': 3} ----------------------------------------
∂f/∂x = 0.24074074074074073
∂f/∂y = -0.16049382716049382
